# Transition system

Let's warm-up by coding a simple transition system in python.
We begin by thinking about it in abstract terms: "What is a transition system?"

We can express this in python by defining a generic interface.
_Note_: clearly this is not the only way, the beauty of programming

In [ ]:
from abc import abstractmethod, ABC
from math import inf
from typing import TypeVar, Generic

X = TypeVar("X")
"""Generic type for the state"""
U = TypeVar("U")
"""Generic type for the input"""


class TransitionSystem(Generic[X, U], ABC):

    @abstractmethod
    def transition(self, x: X, u: U) -> X:
        pass


## Crossing an intersection

Let us consider the most high-level transition system possible to cross an intersection.


* What is a possible choice of state space? action space? transition system?

_Note_: learn to decouple particular implementation choices (e.g., here the state expressed as an `Enum`) from the more abstract interfaces of the system (e.g., transition as a map from state/action to a new state)


In [ ]:
from enum import unique, IntEnum


@unique
class IntState(IntEnum):
    BEFORE = -1
    IN = 0
    AFTER = 1


@unique
class IntAction(IntEnum):
    WAIT = 0
    GO = 1


class IntersectionTS(TransitionSystem[IntState, IntAction]):

    def transition(self, x: IntState, a: IntAction) -> IntState:
        return IntState(min(x + a, IntState.AFTER))


int_ts = IntersectionTS()


## Costs

Let us define a **cost** for the transition system. 
What is a cost?


In [ ]:
class IncrementalCost(Generic[X, U], ABC):

    @abstractmethod
    def cost(self, x: X, a: U, x_new: X) -> float:
        pass


We want to favor plans that take **less time** 

In [ ]:
class IntersectionCost(IncrementalCost[IntState, IntAction]):
    def cost(self, x: IntState, a: IntAction, x_new: IntState) -> float:
        return 1 - int(a)


int_cost = IntersectionCost()

## Planning (without knowing yet how to...)

Now let's find the path with the least cost by checking them all, at random

In [ ]:
from dataclasses import dataclass
from random import choice
from typing import Sequence, AbstractSet, Optional


@dataclass(frozen=True)
class PlannerParams:
    max_search_length: int = 4
    max_attempts: int = 3


class PlannerTransitionSystem(Generic[X, U]):
    def __init__(self, *,
                 x0: X,
                 goal: X,
                 actionset: AbstractSet[U],
                 ts: TransitionSystem,
                 cost: IncrementalCost,
                 params: PlannerParams = PlannerParams()):
        self.x0 = x0
        self.goal = goal
        self.actionset: Sequence[U] = tuple(actionset)
        self.ts = ts
        self.cost = cost
        self.p = params

    def solve(self) -> tuple[float, Sequence[tuple[X, Optional[U]]]]:
        best_path: Sequence[tuple[X, Optional[U]]] = []
        best_cost: float = inf

        for _ in range(self.p.max_attempts):
            path = []
            cost = 0
            x = self.x0
            max_search_length = self.p.max_search_length
            while x != self.goal and max_search_length > 0:
                action = choice(self.actionset)
                path.append((x, action))

                x_new = self.ts.transition(x, action)
                cost += self.cost.cost(x, action, x_new)

                x = x_new
                max_search_length -= 1
            path.append((x, None))

            if x == self.goal and cost < best_cost:
                best_path = path
                best_cost = cost
        return best_cost, best_path


Let's see if it can find a solution for the intersection problem...

In [ ]:

int_planning_problem = PlannerTransitionSystem(
        x0=IntState.BEFORE,
        goal=IntState.AFTER,
        actionset=frozenset([IntAction.GO, IntAction.WAIT]),
        ts=int_ts,
        cost=int_cost)

int_planning_problem.solve()

## Controlling more than 1 agent

Imagine now you can control n>1 robots at the intersection, how would we modify the transition system above?

* What is the state? What are the actions?
* Do we have new costs or constraints?